In [1]:
import numpy as np
from scipy.integrate import odeint
from scipy.optimize import linprog
from matplotlib import pyplot as plt
%matplotlib inline
from tqdm import tqdm

In [2]:
def get_steady_state(A, i, u, res):
    if i == 0:
        return 1.0 / len(A) * np.ones(len(A))
    else:
        deltaA = res.x.reshape(len(A), len(A))
        
        V = np.diag(np.dot(A, u))
        U = np.diag(u)
        invA = np.linalg.inv(A) 
        invUAV = np.linalg.inv(np.dot(U, A) + V)
        
        s1 = -res.fun * np.dot(np.dot(invUAV, invA), np.ones(len(A)))
        s2 = np.dot(np.dot(np.dot(invUAV, U), deltaA), u)
        s3 = np.dot(np.dot(np.dot(np.dot(np.dot(invUAV, invA), deltaA), U), A), u)
                    
        deltaU = s1 - s2 - s3
                    
        return u + np.array(deltaU)

In [3]:
def calc_fitness(A, u):
    AU_sq = np.dot(np.dot(A, np.diag(u)), np.dot(A, np.diag(u)))
    return np.dot(AU_sq, np.ones(len(A)))[0]

In [4]:
def fun(s, t):
    global A 
    AU_sq = np.dot(np.dot(A, np.diag(s)), np.dot(A, np.diag(s)))
    AU_sq_I = np.dot(AU_sq, np.ones(len(A)))  
    return s * (AU_sq_I - np.dot(AU_sq_I, s))

In [5]:
def linprogTask(A, u, constr, EPS, task = 'MAX'):
    
    V = np.diag(np.dot(A, u))
    U = np.diag(u)
    invA = np.linalg.inv(A) 
    invUAV = np.linalg.inv(np.dot(U, A) + V)
    denominator = np.dot(np.dot(np.dot(invUAV, invA), np.ones(len(A))), np.ones(len(A)))
    
    # Коэффициенты в уравнении, которое оптимизируем
    c = []
    for i in range(len(A)):
        for j in range(len(A)):
            
            s = 0 
            for k in range(len(A)):
                s += u[j] * invUAV[k][i] * u[i]
                s1 = 0
                for m in range(len(A)):
                    s2 = 0
                    for l in range(len(A)):
                        s2 += invUAV[k][l] * u[j] * A[j][m] * invA[l][i]
                    s1 += u[m] * s2
                s += s1
                        
            c.append(s / float(denominator))
    
    # Ограничения на delta a_ij 
    bounds = []
    for i in range(len(A)**2):
        bounds.append((-constr, constr))
        
    # Ограничения на сумму a_ij * delta a_ij   
    a = list(np.array(A).reshape(1, len(A)**2).tolist())
    b = [0]
    
    # Условия для принадлежности частот единичному симплексу
    for s in range(len(A)):
        a_ = []; a__ = []
        for i in range(len(A)):
            for j in range(len(A)):
                s1 = 0
                s2 = u[j] * invUAV[s][i] * u[i]
                s3 = 0
                
                for k in range(len(A)):
                    s4 = 0
                    for m in range(len(A)):
                        s4 += invUAV[s][m] * invA[m][i]
                    s3 += u[k] * A[j][k] * u[j] * s4
                       
                for k in range(len(A)):
                    for m in range(len(A)):
                        s1 += invUAV[s][m] * invA[m][k]
                s1 = s1 * c[i * len(A) + j]

                a_.append(s1 - s2 - s3)
                a__.append(-s1 + s2 + s3)  
            
        a.append(a_)
        b.append(1 - u[i] - EPS)
        a.append(a__)
        b.append(u[i] - EPS)
   
    if task == 'MAX':
        res = linprog(np.array(c) * (-1), 
                      A_ub = a, b_ub = b, 
                      bounds = bounds, 
                      method = 'simplex')
    else:
        res = linprog(c, 
                      A_ub = a, b_ub = b, 
                      bounds = bounds, 
                      method = 'simplex')
        
    return res

In [6]:
def plot_steady_state(steadyState):
    plt.figure(figsize = (12, 7))
    
    for i in range(len(steadyState[0])):
        plt.plot(np.array(steadyState)[:, i], label = 'u' + str(i + 1))
        
    plt.legend()
    plt.xlabel(u'Номер итерации', fontsize = 16) 
    plt.ylabel(u'Частота', fontsize = 16)
    plt.title(u'Изменение неподвижной точки', fontsize = 16)
    plt.xticks(fontsize = 16)
    plt.yticks(fontsize = 16)
    plt.show()

In [7]:
def plot_fitness(fitness):
    plt.figure(figsize = (12, 7))
    
    plt.plot(fitness, 'r', label = 'Средний фитнес в стац. состоянии')
        
    plt.legend()
    plt.xlabel(u'Номер итерации', fontsize = 16) 
    plt.ylabel(u'Фитнес', fontsize = 16)
    plt.title(u'Изменение среднего фитнеса', fontsize = 16)
    plt.xticks(fontsize = 16)
    plt.yticks(fontsize = 16)
    plt.show()

In [8]:
def plot_ode(u_ode, t, timeFitnessIntegral):
    n = len(u_ode) // 2 + len(u_ode) % 2 
    
    for i in range(n):
        fig, axs = plt.subplots(figsize = (20, 5), nrows = 1, ncols = 2)
        
        for j in range(len(u_ode[2 * i][0])):
            axs[0].plot(t, np.array(u_ode[2 * i])[:, j], label = 'u' + str(j + 1))
            
        axs[0].legend()
        axs[0].set_xlabel(r"$t$", fontsize = 16) 
        axs[0].set_ylabel(u'Частота', fontsize = 16)
        axs[0].set_title(u'Изменение в динамике на ' + str(timeFitnessIntegral[2 * i]) +\
                         u' итерации', fontsize = 16)
        
        plt.setp(axs[0].get_xticklabels(), fontsize = 16) 
        plt.setp(axs[0].get_yticklabels(), fontsize = 16) 
          
        if (i < n - 1) or (len(u_ode) % 2 == 0):    
            for j in range(len(u_ode[2 * i + 1][0])):
                axs[1].plot(t, np.array(u_ode[2 * i + 1])[:, j], label = 'u' + str(j + 1))



            axs[1].legend()
            axs[1].set_xlabel(r"$t$", fontsize = 16)  
            axs[1].set_ylabel(u'Частота', fontsize = 16)
            axs[1].set_title(u'Изменение в динамике на ' + str(timeFitnessIntegral[2 * i + 1]) +\
                             u' итерации', fontsize = 16)
            plt.setp(axs[1].get_xticklabels(), fontsize = 16) 
            plt.setp(axs[1].get_yticklabels(), fontsize = 16)         

    
        plt.show()

In [19]:
count_iter = 500      # number of iterations of evolution
t1 = 5000            # final point in time to solve the ODE
point_cnt = 10000    # count point for solving ODE
solve_step = 150     # ODU decision step (at which iterations of evolution we will solve an ODE)
EPS = 0.001

n = 5
constr = 0.001

row = [0] * (n - 1) + [1]
A = [list(np.roll(row, i)) for i in range(n)]

u0 = np.random.uniform(size = n)
u0 /= u0.sum()

t = np.linspace(0, t1, point_cnt)

In [ ]:
steadyState = []
fitness = []
norma = []
u_ode = []
timeFitnessIntegral = []

# At each next step, we again find the frequencies and fitness, solve the LPP and get a new matrix A 
for i in tqdm(range(count_iter + 1)):
    
    if i == 0:
        steadyState.append(get_steady_state(A, i, 0, 0))                         # Calculated steady state
    else:
        steadyState.append(get_steady_state(A, i, steadyState[-1], res))         # Calculated steady state
        A = (np.array(A).reshape(1, len(A)**2) + res.x).reshape(len(A), len(A))
    
    fitness.append(calc_fitness(A, steadyState[-1]))                       # Calculated fitness in steady state     
    norma.append(sum(sum(np.array(A) * np.array(A))))                      # Calculated matrix norm

    if (i == 0) or (i == count_iter) or (i % solve_step == 0):
        u_ode.append(odeint(fun, u0, t))                                   # Calculated ODE
        timeFitnessIntegral.append(i)
    
    res = linprogTask(A, steadyState[-1], constr, EPS, task = 'MAX')       # Calculated LPP

 58%|██████████████████████████████████████████████▋                                 | 292/501 [00:10<00:08, 24.93it/s]

In [ ]:
plot_steady_state(steadyState)

In [ ]:
plot_fitness(fitness)

In [ ]:
plot_ode(u_ode, t, timeFitnessIntegral)